<a href="https://colab.research.google.com/github/Athugodage/Summarization_LEA/blob/main/good_GPT_RUSsummarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets tokenizers Sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 36.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 65.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 51.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

In [ ]:
!pip install hugging_face_hub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement hugging_face_hub (from versions: none)
ERROR: No matching distribution found for hugging_face_hub


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
from datasets import load_dataset

data = load_dataset('IlyaGusev/gazeta')

#xsum = load_dataset('xsum')

#data = load_dataset('multi_news')

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/60964 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/6793 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/6369 [00:00<?, ? examples/s]

Dataset gazeta downloaded and prepared to /root/.cache/huggingface/datasets/IlyaGusev___gazeta/default/2.0.0/e2d171980aa248bc22e0af4f8485ad69071fc8e5f3d54a253c71eb434f6694bd. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
from transformers import AutoTokenizer, Trainer, TrainingArguments, AutoModelWithLMHead

model_checkpoint = 'ai-forever/rugpt3small_based_on_gpt2'

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelWithLMHead.from_pretrained(model_checkpoint)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.9/dist-packages/transformers/models/auto/modeling_auto.py:1322: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


In [ ]:
data


DatasetDict({
    train: Dataset({
        features: ['text', 'summary', 'title', 'date', 'url'],
        num_rows: 60964
    })
    test: Dataset({
        features: ['text', 'summary', 'title', 'date', 'url'],
        num_rows: 6793
    })
    validation: Dataset({
        features: ['text', 'summary', 'title', 'date', 'url'],
        num_rows: 6369
    })
})

In [ ]:


max([len(d.split(' ')) for d in data['train']['text']])

1315

In [ ]:
max([len(d.split(' ')) for d in data['train']['summary']])

78

In [ ]:
import torch


tokenizer.add_special_tokens({'pad_token': '[PAD]'})

def tokenize_function(examples):
    law_tokens = tokenizer(examples["text"], truncation=True, max_length=800, padding='max_length', return_special_tokens_mask=True)
    special_tok = tokenizer(" [SEP] ", return_special_tokens_mask=True)
    commentary_tokens = tokenizer(examples["summary"], truncation=True, max_length=50, padding='max_length', return_special_tokens_mask=True)
    tokens = {key: torch.cat((torch.tensor(law_tokens[key]), torch.tensor(special_tok[key]), torch.tensor(commentary_tokens[key])), dim=-1) for key in law_tokens}
    return tokens

In [ ]:
import torch
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import TrainingArguments, Trainer



max_input_length = 800
max_target_length = 50


def preprocess_function(examples):
    inputs = [doc for doc in examples["text"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, padding='max_length')

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["summary"], max_length=max_target_length, truncation=True,padding='max_length')

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs



# Convert pandas DataFrames to Hugging Face Dataset objects
# Tokenize the train and validation datasets
ds = data['train'].map(preprocess_function, batched=True, remove_columns=['text', 'summary', 'title', 'date', 'url'])
val = data['validation'].map(preprocess_function, batched=True, remove_columns=['text', 'summary', 'title', 'date', 'url'])




Map:   0%|          | 0/60964 [00:00<?, ? examples/s]

/usr/local/lib/python3.9/dist-packages/transformers/tokenization_utils_base.py:3596: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/6369 [00:00<?, ? examples/s]

In [ ]:
from transformers import DataCollatorForLanguageModeling
# Set up the data collator
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)


In [ ]:
import numpy as np

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

In [ ]:
!pip install rouge_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24954 sha256=4fb61104d8c9664ce937b12e1740224284a09e5f85b2e81fc396d7580004e8f0
  Stored in directory: /root/.cache/pip/wheels/9b/3d/39/09558097d3119ca0a4d462df68f22c6f3c1b345ac63a09b86e
Successfully built rouge_score


In [ ]:
from datasets import load_metric


rouge = load_metric("rouge")

def compute_metrics(eval_preds):
    inputs, preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)


    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    decoded_inputs = tokenizer.batch_decode(inputs, skip_special_tokens=True)

    # Some simple post-processing
    decoded_inputs, decoded_preds, decoded_labels = postprocess_text(decoded_inputs, decoded_preds,
                                                                        decoded_labels, "rouge")
    rouge_result = rouge._compute(inputs=decoded_inputs, predictions=decoded_preds, references=decoded_labels)

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="Russian_GPT_summarizer",
    num_train_epochs=1,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=1,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    weight_decay=0.01,
    logging_dir="logs",
    push_to_hub=True)
    

# Set up the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=ds,
    eval_dataset=val,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)



Cloning https://huggingface.co/marcus2000/Russian_GPT_summarizer into local empty directory.


In [ ]:
# Train the model
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: ignored

In [ ]:
trainer.push_to_hub()

Upload file pytorch_model.bin:   0%|          | 1.00/526M [00:00<?, ?B/s]

Upload file training_args.bin:   0%|          | 1.00/3.43k [00:00<?, ?B/s]

To https://huggingface.co/marcus2000/Russian_GPT_summarizer
   5d011f6..9a7fab7  main -> main

   5d011f6..9a7fab7  main -> main

To https://huggingface.co/marcus2000/Russian_GPT_summarizer
   9a7fab7..71eb574  main -> main

   9a7fab7..71eb574  main -> main



'https://huggingface.co/marcus2000/Russian_GPT_summarizer/commit/9a7fab7e45a6774ca86a833efe808d2a6583958a'

In [ ]:
from transformers import pipeline 

model = pipeline('text-generation', model='Russian_GPT_summarizer', tokenizer='ai-forever/rugpt3small_based_on_gpt2')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
data['test']['text'][1]

"Высокопоставленная американская и израильская делегация находятся в ОАЭ с двухдневным визитом, за время которого стороны заключили историческое соглашение о нормализации отношений. «Соглашение, достигнутое между США, Израилем и ОАЭ 13 августа 2020 года, является смелым шагом на пути к более стабильному, объединенному и процветающему Ближнему Востоку. Оно переворачивает традиционное представление о способах решения проблем и задач региона, с упором на прагматические шаги, которые имеют ощутимые практические результаты. Соглашение несет в себе надежду на создание новых мостов, которые будут способствовать деэскалации существующих конфликтов и предотвращению конфликтов в будущем», — говорится в тексте совместного заявления представителей трех стран. Еще одним символическим шагом стал первый прямой авиарейс из Тель-Авива, на котором и прибыли зарубежные гости в ОАЭ. Рейсу был присвоен номер LY971 — международный телефонный код Арабских Эмиратов. Во время визита в Абу-Даби стороны намерены

In [ ]:
model(data['test']['text'][1] + ' [SEP] ', max_length=100)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Input length of input_ids is 1593, but `max_length` is set to 100. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.


[{'generated_text': "Высокопоставленная американская и израильская делегация находятся в ОАЭ с двухдневным визитом, за время которого стороны заключили историческое соглашение о нормализации отношений. «Соглашение, достигнутое между США, Израилем и ОАЭ 13 августа 2020 года, является смелым шагом на пути к более стабильному, объединенному и процветающему Ближнему Востоку. Оно переворачивает традиционное представление о способах решения проблем и задач региона, с упором на прагматические шаги, которые имеют ощутимые практические результаты. Соглашение несет в себе надежду на создание новых мостов, которые будут способствовать деэскалации существующих конфликтов и предотвращению конфликтов в будущем», — говорится в тексте совместного заявления представителей трех стран. Еще одним символическим шагом стал первый прямой авиарейс из Тель-Авива, на котором и прибыли зарубежные гости в ОАЭ. Рейсу был присвоен номер LY971 — международный телефонный код Арабских Эмиратов. Во время визита в Абу-Д